<!-- Assignment 6 - SS 2023 -->

# Adversarial Training (15 points)

This notebook contains one of the assignments for the exercises in Deep Learning and Neural Nets 2.
It provides a skeleton, i.e. code with gaps, that will be filled out by you in different exercises.
All exercise descriptions are visually annotated by a vertical bar on the left and some extra indentation,
unless you already messed with your jupyter notebook configuration.
Any questions that are not part of the exercise statement do not need to be answered,
but should rather be interpreted as triggers to guide your thought process.

**Note**: The cells in the introductory part (before the first subtitle)
perform all necessary imports and provide utility functions that should work without (too much) problems.
Please, do not alter this code or add extra import statements in your submission, unless explicitly allowed!

<span style="color:#d95c4c">**IMPORTANT:**</span> Please, change the name of your submission file so that it contains your student ID!

In this assignment, we will play around with adversarial techniques in deep learning.
The main goal is to build some understanding of *Generative Adversarial Networks* (GANs).
However, we will also take a look at *adversarial training*,
which is a collection of methods for fooling neural networks.
Although both approaches are completely unrelated,
it turns out that is possible to build GANs, starting from adversarial examples.


In [ ]:
import os

import torch
from torch import nn
from torch import optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from torchvision import datasets
from PIL import Image

torch.manual_seed(1806)
torch.cuda.manual_seed(1806)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

In [ ]:
# google colab data management
import os.path

try:
    from google.colab import drive
    drive.mount('/content/gdrive')
    _home = 'gdrive/MyDrive/'
except ImportError:
    _home = '~'
finally:
    data_root = os.path.join(_home, '.pytorch')

print(data_root)

In [ ]:
class BaseTrainer:

    def __init__(self,
         model: nn.Module,
         criterion: nn.Module,
         optimiser: optim.Optimizer,
    ):
        """
        Parameters
        ----------
        model : torch.nn.Module
            Neural Network that will be trained.
        criterion : torch.nn.Module
            Loss function to use for training.
        optimiser : torch.optim.Optimizer
            Optimisation strategy for training.
        tracker : Tracker, optional
            Tracker to keep track of training progress.
        """
        self.model = model
        self.criterion = criterion
        self.optimiser = optimiser

    def state_dict(self):
        """ Current state of learning. """
        return {
            "model": self.model.state_dict(),
            "objective": self.criterion.state_dict(),
            "optimiser": self.optimiser.state_dict(),
        }
    
    def load_state_dict(self, state_dict: dict):
        """ Restore learning state. """
        self.model.load_state_dict(state_dict["model"])
        self.criterion.load_state_dict(state_dict["objective"])
        self.optimiser.load_state_dict(state_dict["optimiser"])

    @property
    def device(self):
        """ Device of the (first) model parameters. """
        return next(self.model.parameters()).device

    def _forward(self, data: DataLoader, metric: callable):
        device = self.device
        for x, y in data:
            x, y = x.to(device), y.to(device)
            outputs = self.model(x)
            res = metric(outputs, y)
            yield res

    @torch.no_grad()
    def evaluate(self, data: DataLoader, metric: callable) -> list:
        self.model.eval()

        results = self._forward(data, metric)
        return [res.item() for res in results]


    @torch.enable_grad()
    def update(self, data: DataLoader) -> list:
        opt = self.optimiser
        self.model.train()

        errs = []
        for err in self._forward(data, self.criterion):
            errs.append(err.item())

            opt.zero_grad()
            err.backward()
            opt.step()

        return errs
    
    @property
    def fake_data(self):
        return None
        
    
    def _display_results(self, count: int = 10):
        """
        Display generated images.

        Parameters
        ----------
        count : int, optional
            Number of images to display.
        """
        fake_data = self.fake_data
        if fake_data is None:
            return
        
        x = fake_data[:count].to(self.device)
        ps = self.model.probabilities(x).view(-1).tolist()
        print("D probs:", ', '.join([f"{p:.2f}" for p in ps]))

        im = data_to_image(*x.cpu(), means=(.1307, ), stds=(.3081, ))
        display(im, metadata={'width': '100%'})


    def train(self, loader: DataLoader, num_epochs: int = 10, vis_every: int = 5):
        """
        Train an auto-encoder for a number of epochs.

        Parameters
        ----------
        loader : DataLoader
            A data loader for iterating over batches of the data.
        num_epochs : int, optional
            Number of times to iterate the dataset.
        vis_every : int, optional
            Frequency, during training, of 
            intermediate visualisation of faked samples.
        """
        # evaluate random performance
        errs = self.evaluate(loader, self.criterion)
        print(f"Epoch {0: 2d} - avg loss: {sum(errs) / len(errs):.6f}")
        self._display_results()

        # train for some epochs
        for epoch in range(1, num_epochs + 1):
            errs = self.update(loader)
            print(f"Epoch {epoch: 2d} - avg loss: {sum(errs) / len(errs):.6f}")

            if epoch % vis_every == 0:
                self._display_results()
        
        return errs

In [ ]:
@torch.no_grad()
def accuracy(logits, targets):
    """
    Compute the accuracy for given logits and targets.
    
    Parameters
    ----------
    logits : (N, K) torch.Tensor
        A mini-batch of logit vectors from the network.
    targets : (N, ) torch.Tensor
        A mini_batch of target scalars representing the labels.
        
    Returns
    -------
    acc : () torch.Tensor
        The accuracy over the mini-batch of samples.
    """
    correct = torch.argmax(logits, dim=-1) == targets
    acc = torch.mean(correct.float())
    return acc

In [ ]:
def data_to_image(*data: torch.Tensor,
                  means: tuple = (0, ), stds: tuple = (1., )) -> Image:
    """
    Convert multiple tensors to one big image.

    Parameters
    ----------
    data0, data1, ... dataN : torch.Tensor
        One or more tensors to be merged into a single image.
    means : tuple or torch.Tensor, optional
        Original mean of the image before normalisation.
    stds : tuple or torch.Tensor, optional
        Original standard deviation of the image before normalisation.

    Returns
    -------
    image : Image
        PIL image with all of the tensors next to each other.
    """
    # concatenate all data
    big_pic = torch.cat([x for x in data], dim=-1)

    means = torch.tensor(means)
    stds = torch.tensor(stds)
    to_image = transforms.Compose([
        # inverts normalisation of image
        transforms.Normalize(-means / stds, 1. / stds),
        transforms.Lambda(lambda x: torch.clamp(x, 0, 1)),
        transforms.ToPILImage()
    ])

    return to_image(big_pic)

def display_fakes(discriminator: nn.Module, fake_data: torch.Tensor,
                  count: int = 10):
    """
    Display the images that are fed as fakes to a discriminator.

    Parameters
    ----------
    discriminator : nn.Module
        The discriminator that is to be fooled by the fakes.
    fake_data : (N, C, H, W) torch.Tensor
        The fake images to display.
    count : int, optional
        Number of samples to display from `fake_data`.
    """
    device = next(discriminator.parameters()).device
    x = fake_data[:count].to(device)
    ps = discriminator.probabilities(x).view(-1).tolist()
    print("D probs:", ', '.join([f"{p:.2f}" for p in ps]))
    
    im = data_to_image(*x.cpu(), means=(.1307, ), stds=(.3081, ))
    display(im, metadata={'width': '100%'})

In [ ]:
def evaluate_attack(attack: callable, network: nn.Module, data: DataLoader,
                    loss_func: nn.Module, vis_count: int = 8, **kwargs):
    """
    Evaluate an adversarial attack on some data.

    Parameters
    ----------
    attack : callable
        Function that computes the adversarial example.
        Will be called as `attack(x, y, network, loss_func, **kwargs).
    network : nn.Module
        The network to attack.
    data : DataLoader
        Data for the attack.
    loss_func : nn.Module
        Objective for the attack.
    vis_count : int, optional
        Number of adversarial examples to visualise.
    kwargs
        Keyword arguments for the attack function.
    """
    dev = next(network.parameters()).device
    network.eval()

    accuracies = []
    for x, y in data:
        x, y = x.to(dev), y.to(dev)
        adv_x = attack(x, y, network, loss_func, **kwargs)
        adv_x = adv_x.clamp(x.min(), x.max())  # put in valid range

        # compute accuracy on adversarial examples
        with torch.no_grad():
            logits = network(adv_x)
            acc = accuracy(logits, y)
            accuracies.append(acc.item())

    print(f"Adversarial accuracy: {100 * accuracies[0]:.2f}%")

    # collect relevant data
    x_ref, y_ref = x[:vis_count], y[:vis_count]
    x_adv, y_adv = adv_x[:vis_count], logits[:vis_count].argmax(-1)
    with torch.no_grad():
        y_pred = network(x_ref).argmax(-1)
    
    # visualise adversaries
    ref_im = data_to_image(*x_ref.cpu(), means=(.1307, ), stds=(.3081, ))
    display(ref_im, metadata={'width': '100%'})
    print("ground truth: ", y_ref.tolist())
    print("before attack:", y_pred.tolist())
    print("after attack: ", y_adv.tolist())
    adv_im = data_to_image(*x_adv.cpu(), means=(.1307, ), stds=(.3081, ))
    display(adv_im, metadata={'width': '100%'})

In [ ]:
class DiscriminatorTrainer(BaseTrainer):
    
    def __init__(self, model, criterion, optimiser,
                 dataset: "RealOrFakeDataset", epsilon: float = 0.1):
        """
        Parameters
        ----------
        dataset : RealOrFakeDataset
            The dataset that will be used to train the discriminator.
        epsilon: float, optional
            The maximum allowed perturbation for the adversarial attacks.
        """
        super().__init__(model, criterion, optimiser)
        self.dataset = dataset
        self.epsilon = epsilon
    
    @property
    def fake_data(self):
        return self.dataset.fake_data
    
    def update(self, data: DataLoader) -> list:
        results = super().update(data)
        self.dataset.update_fake_examples(self.model, self.criterion, self.epsilon)
        return results
        

def train_gan(gan: nn.Module, loader: DataLoader, objective: nn.Module, 
              optimiser: optim.Optimizer, num_epochs: int = 10, vis_every: int = 5):
    """
    Train a GAN for a number of epochs.
    
    Parameters
    ----------
    gan : nn.Module
        The GAN to train
    loader : DataLoader
        A data loader for iterating over batches of the data.
        This can be a standard, "supervised" dataloader.
    objective : nn.Module
        The loss function to optimise during training.
    optimiser : optim.Optimizer
        The optimiser to use for training.
    num_epochs : int, optional
        Number of times to iterate the dataset.
    vis_every : int, optional
        Frequency, during training, of 
        intermediate visualisation of fake samples.
        
    """
    g_errs, d_errs = [], []
    for x, _ in loader:
        x = x.to(device)
        g_errs.append(generator_error(gan, x, objective).item())
        d_errs.append(discriminator_error(gan, x, objective).item())
    loss_report = ', '.join([
        f"avg G loss: {sum(g_errs) / len(g_errs):.5f}",
        f"avg D loss: {sum(d_errs) / len(d_errs):.5f}"
    ])
    print(f'Epoch {0: 2d}', loss_report, sep=' - ')
    fake_data = gan.generator.generate(batch_size=10)
    display_fakes(gan.discriminator, fake_data)
    
    for epoch in range(1, num_epochs + 1):
        g_errs, d_errs = update_gan(gan, loader, objective, optimiser)
        loss_report = ', '.join([
            f"avg G loss: {sum(g_errs) / len(g_errs):.5f}",
            f"avg D loss: {sum(d_errs) / len(d_errs):.5f}"
        ])
        print(f'Epoch {epoch: 2d}', loss_report, sep=' - ')
        if epoch % vis_every == 0:
            fake_data = gan.generator.generate(batch_size=10)
            display_fakes(gan.discriminator, fake_data)

## Adversarial Examples

In [ ]:
# set up training (nothing to do here)

pre_processing = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((.1307, ), (.3081, ))
])
mnist_train = datasets.MNIST(data_root, transform=pre_processing)
train_loader = DataLoader(mnist_train, shuffle=True, batch_size=128, num_workers=4)

mnist_net = nn.Sequential(
    nn.Conv2d(1, 8, 4, stride=2),
    nn.ELU(),
    nn.Conv2d(8, 32, 3, stride=2),
    nn.ELU(),
    nn.Conv2d(32, 64, 6),
    nn.ELU(),
    nn.Flatten(),
    nn.Linear(64, 10)
).to(device)

trainer = BaseTrainer(
    model=mnist_net,
    criterion=nn.CrossEntropyLoss(),
    optimiser=optim.SGD(mnist_net.parameters(), lr=1e-2, momentum=.9)
)

In [ ]:
%%time

# train MNIST classifier
errs = trainer.train(train_loader, num_epochs=20)
errs = trainer.evaluate(train_loader, accuracy)
print(f"Train accuracy: {100 * sum(errs) / len(errs):.2f}%")
torch.save(trainer.state_dict(), "mnist_net.pt")  # for testing
print()

In [ ]:
# test MNIST classifier
trainer.load_state_dict(torch.load("mnist_net.pt"))  # for testing
mnist_test = datasets.MNIST(data_root, train=False, transform=pre_processing)
test_loader = DataLoader(mnist_test, batch_size=len(mnist_test), num_workers=4)
errs = trainer.evaluate(test_loader, trainer.criterion)
print(f"Eval loss: {errs[0]:.5f}")
errs = trainer.evaluate(test_loader, accuracy)
print(f"Eval accuracy: {100 * errs[0]:.2f}%")

It has become extremely easy to get good models for plenty of tasks.
It only takes a few lines of code and a handful of minutes
to get a model that is able to recognise handwritten digits.
This does not mean, however, that the network *understands* these images.
It turns out that even the best models can easily be fooled
by creating what is known as *adversarial examples*.

There are mainly two ways to make a strong network look stupid:
 1. Change existing images until the network mis-classifies them,
 2. Make predictions on random noise (out-of-distribution images).

The first approach is an active topic in Machine Learning research.
The main goal is to find the smallest possible perturbations
that are required to make the network misclassify its inputs.

The second approach can be considered silly because in most cases, 
a network is simply not able to express that it has no idea what to predict.
Nevertheless, this can make systems that rely on neural networks vulnerable.
E.g. face recognition software being fooled by carefully constructed noise.

### Exercise 1: Projected Gradient Ascent (5 points)

One of the easiest ways to construct an adversarial example
is to maximise the loss by using gradient ascent on the inputs.
After all, the parameters of a neural network are also the result
of a gradient descent that minises the loss function.

In order to keep the perturbations to the original image small,
a constraint is added to the gradient ascent for adversarial examples:
the adversarial example must lie in an epsilon-ball around the original input.
This procedure is also known as *projected gradient ascent*.

 > Implement the `adversarial_attack` function below
 > so that it implements projected gradient ascent as described above.
 > To speed up the process of finding adversarial examples,
 > you can use the sign of the gradients instead of the pure gradients.
 > Make sure not to return the computational graph!
 
**Hint:** If you are having problems with gradient errors, your computations might involve an unwanted gradient flow.

In [ ]:
def adversarial_attack(x: torch.Tensor, y: torch.Tensor,
                       network: nn.Module, loss_func: nn.Module,
                       epsilon: float = .1, eta: float = .1, steps: int = 10):
    """
    Compute an adversarial example for a network from a given input.

    Parameters
    ----------
    x : torch.Tensor
        The reference input for the network.
    y : torch.Tensor
        The label for the reference input.
    network : nn.Module
        The network to compute the adversary for.
    loss_func : nn.Module
        The loss function the network was trained with.
    epsilon : float, optional
        The radius of the ball to keep the adversary within.
    eta : float, optional
        The step size to use for computing the adversary.
    steps : int, optional
        The number of gradient steps to take.

    Returns
    -------
    adversary : torch.Tensor
        The adversarial example (`x + perturbation`).
    """
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# Test Cell: do not edit or delete!
x = torch.ones(1, 1, 28, 28).to(trainer.device)
y = torch.zeros(1, dtype=torch.long).to(trainer.device)
x_adv = adversarial_attack(x, y, trainer.model, trainer.criterion, epsilon=0.1)
assert x.shape == x_adv.shape, (
    "ex1: adversarial_attack produces outputs with incorrect shape (-1 point)"
)
assert torch.abs(x - x_adv).max() <= 0.1 + 1e-5, (
    "ex1: adversarial_attack produces outputs outside of epsilon ball (-1 point)"
)

In [ ]:
# Test Cell: do not edit or delete!
assert trainer.model(x_adv)[:, y] < trainer.model(x)[:, y], (
    "ex1: adversarial_attack produces outputs with better results (-1 point)"
)

In [ ]:
# Test Cell: do not edit or delete!
assert len(torch.abs(x - x_adv).unique()) <= 5, (
    "ex1: adversarial_attack does not seem to change pixels fast enough (-1 point)"
)

In [ ]:
# Test Cell: do not edit or delete!

In [ ]:
# Test Cell: do not edit or delete!

In [ ]:
# Test Cell: do not edit or delete!

In [ ]:
# sanity check
trainer.model.requires_grad_(False)  # disable parameter gradients
evaluate_attack(adversarial_attack, trainer.model, test_loader, trainer.criterion, epsilon=.3)

## Fooling a Dummy Network

Adversarial examples are considered an analysis tool
that can help to gain insights in how neural networks behave.
However, when pushing these methods, it turns out that
adversarial examples can actually also be used to generate new images.
The key point to getting useful images, is to attack the right model.

Consider the task of predicting whether an input is *real* or *fake*.
A *real* sample would be defined as an input that comes from the dataset,
whereas a *fake* example is an input that does not appear in the dataset.
It should be relatively easy to build a binary classification model for this task.
Let us denote this model as a *discriminator*,
since it discriminates between *real* and *fake* images.

By using an adversarial attack on this model 
*fake* inputs can be altered to look more like *real* samples.

In [ ]:
class Discriminator(nn.Module):
    """ Network for deciding whether a MNIST image is real or fake. """
    
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Conv2d(1, 8, 4, stride=2),
            nn.ELU(),
            nn.Conv2d(8, 32, 3, stride=2),
            nn.ELU(),
            # (old-school) global average pooling
            nn.Conv2d(32, 64, 6),
            nn.ELU(),
            nn.Flatten(),
            nn.Linear(64, 1)
        )
    
    def forward(self, x):
        return self.model(x)
    
    @torch.no_grad()
    def probabilities(self, x):
        """
        Compute probability of inputs being real.

        Parameters
        ----------
        x : torch.Tensor
            The inputs to compute the probabilities for.

        Returns
        -------
        p : torch.Tensor
            Values between 0 (fake) and 1 (real).

        Notes
        -----
        Gradient computations are disabled.
        """
        logits = self.model(x)
        return torch.sigmoid(logits)
    
    @torch.no_grad()
    def predict(self, x):
        """
        Make a hard prediction on fake or real.

        Parameters
        ----------
        x : torch.Tensor
            The inputs to compute the prediction for.

        Returns
        -------
        y : torch.Tensor
            Either 0 (fake) or 1 (real).

        Notes
        -----
        Gradient computations are disabled.
        """
        logits = self.model(x)
        return (logits > 0).int()

###### Paradox of the Discriminator

If we compute adversarial examples for the discriminator,
we can only get more *fake* inputs,
since the adversaries are not part of the dataset.
Now, a successful adversary of a *real* input,
should be classified as a *fake*, and vice versa.
Note that successful adversaries of *real* inputs 
will be **correctly** classified as *fake*.
Hence, the obtained example would not really be adversarial.
Therefore, the only truly adversarial examples for this model
are *fake* inputs that are converted to be recongised as *real*.

### Exercise 2: Discriminator Data (3 points)

Training a model that is able to discriminate between *real* and *fake*,
is not entirely straightforward.
The main problem is to obtain a reasonable dataset 
that represents both classes equally well.

One solution is to use random noise for the *fake* inputs.
However, this often makes things too easy
and will not produce a particularly strong model.
To counter this issue, we can use an adversarial attack 
to make the *fake* inputs appear more real
after every epoch of discriminator training.
This turns out to produce a reasonable discriminator
as well as reasonably good *fake* inputs.

 > Implement the `update_fake_examples` method of the `RealOrFakeDataset`
 > to implement the approach described above.
 > Make sure to keep the adversarial examples in a valid range!
 
**Hint:** You can use the `adversarial_attack` function from the previous exercise to compute adversarial examples.

In [ ]:
class RealOrFakeDataset(Dataset):
    """ Dataset for enhancing a (small) dataset with fake inputs. """
    
    def __init__(self, real_dataset, seed: int = None):
        super().__init__()
        self.ref_dataset = real_dataset
        
        # control randomness
        rng = torch.Generator()
        if seed is not None:
            rng.manual_seed(seed)
        
        # get real, pre-processed data
        real_data = torch.stack([x for x, _ in real_dataset])
        mean, std = real_data.mean(), real_data.std()
        min_x, max_x = real_data.min(), real_data.max()
        self.min_x, self.max_x = min_x, max_x
        # generate fake data (with similar statistics)
        self.fake_data = torch.empty_like(real_data)
        self.fake_data.normal_(mean, std, generator=rng).clamp_(min_x, max_x)
    
    def __getitem__(self, index):
        new_index = index - len(self.ref_dataset)
        if new_index < 0:
            x, _ = self.ref_dataset[index]
            return x, torch.ones(1)
        
        return self.fake_data[new_index], torch.zeros(1)
    
    def __len__(self):
        return len(self.ref_dataset) + len(self.fake_data)
    
    @property
    def raw_folder(self):
        return os.path.join(self.root, 'MNIST', 'raw')

    @property
    def processed_folder(self):
        return os.path.join(self.root, 'MNIST', 'processed')
    
    def update_fake_examples(self, network: nn.Module, loss_func: nn.Module,
                             epsilon: float = .1):
        """
        Update the fake examples of this dataset.

        The fake examples are updated by computing adversarial examples
        for the network that is trained with the data from this dataset.

        Parameters
        ----------
        network : nn.Module
            Network to use for computing adversarial examples.
        loss_func : nn.Module
            Loss function to use for computing adversarial examples.
        epsilon : float, optional
            Epsilon parameter for the adversarial attack.

        Notes
        -----
        This method overwrites the fake samples.
        It should not be necessary to create a new dataset
        or set up a new data loader.
        """
        device = next(network.parameters()).device
        # YOUR CODE HERE
        raise NotImplementedError()

In [ ]:
# Test Cell: do not edit or delete!
discriminator = Discriminator()
bce = nn.BCEWithLogitsLoss()
data = RealOrFakeDataset(torch.utils.data.Subset(mnist_train, range(64)))

old_samples = data.fake_data.clone()
data.update_fake_examples(discriminator, bce)
new_samples = data.fake_data
assert torch.any(old_samples != new_samples), (
    "ex2: update_fake_examples does nothing (-1 point)"
)

In [ ]:
# Test Cell: do not edit or delete!

In [ ]:
# Test Cell: do not edit or delete!
assert torch.all(discriminator(new_samples) > discriminator(old_samples)), (
    "ex2: update_fake_examples does not make samples more real (-1 point)"
)

In [ ]:
# Test Cell: do not edit or delete!

### Exercise 3: Training the Discriminator (1 point)

Finally, we can use this dataset for training the discriminator.
By updating the *fake* inputs after every epoch of training,
the quality of the *fake* inputs, and therefore also the discriminator,
should improve steadily during training.

Time to test if this actually works.
The `train_discriminator` function (in the preamble)
implements a training loop that updates the fake examples every epoch.
The code below is able to train the discriminator to zero loss.
However, the *fake* images do not look very realistic.

 > Play around with some of the hyperparameters (lr, epsilon, ...)
 > to get more realistic *fake* images,
 > so that the task of the discriminator becomes harder.

In [ ]:
d_data = RealOrFakeDataset(mnist_train)
d_loader = DataLoader(d_data, batch_size=128, shuffle=True)

discriminator = Discriminator().to(device)
trainer = DiscriminatorTrainer(
    model=discriminator,
    criterion=nn.BCEWithLogitsLoss(),
    optimiser=optim.Adam(discriminator.parameters(), lr=1e-2),
    dataset=d_data,
    epsilon=0.1
)

# YOUR CODE HERE
raise NotImplementedError()

_ = trainer.train(d_loader, num_epochs=15, vis_every=5)

## Generative Adversarial Modelling

The previously discussed approach, using adversarial training,
is able to create somewhat realistic inputs.
Although it might be considered a way to generate new images,
it is not very practical because it does not allow 
to generate samples after training.

Although Generative Adversarial Nets (GANs)
do not really have a connection with adversarial examples
(except for the main author and the name),
the goal is also to "fool" a discriminator network.
Instead of using adversarial examples as *false* examples, however,
GANs use a decoder-style network, referred to as *generator*
that maps from a known distribution to the input space.

In [ ]:
class Generator(nn.Module):
    """ Network for creating fake MNIST images. """
    
    def __init__(self, latent_dim: int = 64):
        """
        Parameters
        ----------
        latent_dim : int, optional
            Dimensions of the latent space, from which to sample.
        """
        super().__init__()
        self.latent_dim = latent_dim
        self.model = nn.Sequential(
            nn.ConvTranspose2d(latent_dim, 32, 6),
            nn.ELU(),
            nn.ConvTranspose2d(32, 8, 3, stride=2),
            nn.ELU(),
            nn.ConvTranspose2d(8, 1, 4, stride=2),
        )
        
    def forward(self, z):
        x = self.model(z.view(-1, self.latent_dim, 1, 1))
        return x
        
    @torch.no_grad()
    def sample(self, batch_size: int):
        """
        Sample in the latent space of the generator.

        Parameters
        ----------
        batch_size : int
            Number of samples to sample.

        Returns
        -------
        z : (batch_size, latent_dim) torch.Tensor
            A random vector in the latent space.

        Notes
        -----
        Gradient computations are disabled.
        """
        z = torch.randn(batch_size, self.latent_dim)
        return z.to(self.model[0].weight.device)
    
    @torch.no_grad()
    def generate(self, z = None, batch_size: int = 1, bounded: bool = True):
        """
        Generate one or more random images.

        Parameters
        ----------
        z : (N, latent_dim) torch.Tensor
            Latent vector(s) to use for generation.
        batch_size : int, optional
            Number of images to generate.
            This parameter is ignored if `z` is specified.
        bounded : bool, optional
            Flag to assure that pixel values are in a valid range.

        Returns
        -------
        x : (batch_size, C, H, W) torch.Tensor
            Generated image(s).
        """
        if z is None:
            z = self.sample(batch_size)
        
        raw = self.forward(z)
        
        if bounded:
            raw.clamp_(-0.4242, 2.8215)  # MNIST range
            
        return raw

### Excercise 4: Min-Max Game (6 points)


The goal of the generator is to create images that maximise the loss
that the discriminator attempts to minimise.
The result is a min-max game between generator and discriminator.
Mathematically, this leads to the following optimisation target:
$$\begin{aligned}
\max_d \min_g \mathbb{E}[\log d(X)] + \mathbb{E}[\log(1 - d(g(Z)))].
\end{aligned}$$
Note that this function is simply the negated binary cross-entropy loss.

 > Finish the implementation of the `update_gan` function,
 > including the `generator_error` and `discriminator_error` functions.
 > Make sure that the generator-error is used to update only the generator,
 > i.e. does not affect the discriminator, and vice versa.
 > To achieve this, you will need to control the gradients
 > with the techniques discussed in assignment 1.
 > Also, you do not want to directly maximise the error term for the generator,
 > since it suffers from vanishing gradients.
 > Think about an alternative, but equivalent optimisation problem.
 

In [ ]:
def generator_error(gan: nn.Module, real_x: torch.Tensor, loss_func: nn.Module):
    """
    Construct the gradient graph for updating the generator.

    Parameters
    ----------
    gan : nn.Module
        Module with `generator` and `discriminator` sub-modules.
    real_x: torch.Tensor
        Mini-batch from target distribution.
    loss_func: nn.Module
        Loss function taking logits and target values.

    Returns
    -------
    err : torch.Tensor
        Scalar tensor with graph for updating generator.
    """
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# Test Cell: do not edit or delete!
gan = nn.ModuleDict({
    'generator': Generator(),
    'discriminator': Discriminator()
})
bce = nn.BCEWithLogitsLoss()

try:
    err = generator_error(gan, torch.randn(5, 1, 28, 28), bce)
    grads = torch.autograd.grad(err, gan.generator.parameters())
except RuntimeError:
    raise AssertionError(
        "ex4: generator_error does not allow to compute gradients (-1 point)"
    )

In [ ]:
# Test Cell: do not edit or delete!

In [ ]:
def discriminator_error(gan: nn.Module, real_x: torch.Tensor, loss_func: nn.Module):
    """
    Construct the gradient graph for updating the discriminator.

    Parameters
    ----------
    gan : nn.Module
        Module with `generator` and `discriminator` sub-modules.
    real_x: torch.Tensor
        Mini-batch from target distribution.
    loss_func: nn.Module
        Loss function taking logits and target values.

    Returns
    -------
    err : torch.Tensor
        Scalar tensor with graph for updating discriminator.
    """
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# Test Cell: do not edit or delete!
gan = nn.ModuleDict({
    'generator': Generator(),
    'discriminator': Discriminator()
})
bce = nn.BCEWithLogitsLoss()

try:
    err = discriminator_error(gan, torch.randn(5, 1, 28, 28), bce)
    grads = torch.autograd.grad(err, gan.discriminator.parameters())
except RuntimeError:
    raise AssertionError(
        "ex4: discriminator_error does not allow to compute gradients (-1 point)"
    )

In [ ]:
# Test Cell: do not edit or delete!

In [ ]:
def update_gan(gan: nn.Module, loader: DataLoader, loss_func: nn.Module,
               opt: optim.Optimizer):
    """
    Train GAN for one epoch.

    Parameters
    ----------
    gan : nn.Module
        Module with `generator` and `discriminator` sub-modules.
    loader : DataLoader
        Standard, supervised data loader with samples from target distribution.
    loss_func : nn.Module
        Loss function taking logits and target values.
    opt : optim.Optimizer
        Optimiser for updating GAN parameters.

    Returns
    -------
    g_errs : list
        List of computed errors for generator update in each batch.
    d_errs : list
        List of computed errors for discriminator update in each batch.
    """
    gan.train()
    device = next(gan.parameters()).device
    
    d_errs, g_errs = [], []
    for x, _ in loader:
        x = x.to(device)
        # YOUR CODE HERE
        raise NotImplementedError()

    return g_errs, d_errs

In [ ]:
# Test Cell: do not edit or delete!
gan = nn.ModuleDict({
    'generator': Generator(),
    'discriminator': Discriminator()
}).to(device)
bce = nn.BCEWithLogitsLoss()
opt = optim.Adam(gan.parameters(), lr=1e-3)

old_pars = {k: v.clone() for k, v in gan.state_dict().items()}
g_errs, d_errs = update_gan(gan, [next(iter(train_loader))], bce, opt)
assert len(g_errs) == len(test_loader), (
    "ex4: wrong number of generator errors in update_gan(-1 point)"
)
assert len(d_errs) == len(test_loader), (
    "ex4: wrong number of discriminator errors in update_gan (-1 point)"
)

In [ ]:
# Test Cell: do not edit or delete!
assert all(
    torch.any(old_pars[k] != par) for k, par in gan.state_dict().items()
), "ex4: update_gan does not update all parameters (-1 points)"

In [ ]:
# Test Cell: do not edit or delete!

In [ ]:
# sanity check
gan = nn.ModuleDict({
    'generator': Generator(),
    'discriminator': Discriminator()
}).to(device)
bce = nn.BCEWithLogitsLoss()
opt = optim.Adam([
    {'params': gan.generator.parameters(), 'lr': 1e-3},
    {'params': gan.discriminator.parameters(), 'lr': 2e-4}
])
train_gan(gan, train_loader, bce, opt, num_epochs=25)